# Load data

In [1]:
import os

# load simulated data
from joblib import load

all_states = load("simulated_states.joblib")
all_labels = load("simulated_labels.joblib")

b# Preprocessingb

In [2]:
# get dataframe
import pandas as pd

# column titles
state_columns = [
    "agent_x", "agent_y",
    "target_x", "target_y",
    "at_up", "at_down", "at_left", "at_right",
    "hasItem", "energy", "charge"
]

# Create DataFrame
df = pd.DataFrame(all_states, columns=state_columns)
df.head()

,agent_x,agent_y,target_x,target_y,at_up,at_down,at_left,at_right,hasItem,energy,charge
0,4,4,11,2,0,0,4,4,0,119,0
1,1,1,6,5,2,0,0,0,0,119,0
2,9,7,3,2,0,0,4,4,0,119,0
3,5,4,11,2,0,0,4,4,0,118,0
4,2,1,6,5,0,0,4,4,0,118,0


In [19]:
# one hot encoding
from joblib import dump
from sklearn.preprocessing import OneHotEncoder

neighbour_col = ["at_up", "at_down", "at_left", "at_right"]

# Explicitly define all categories for consistency
all_possible_categories = [[-1, 0, 1, 2, 3, 4, 5]] * len(neighbour_col)

encoder = OneHotEncoder(
    categories=all_possible_categories,
    handle_unknown="ignore",
    sparse_output=False
).set_output(transform="pandas")

transformed_columns = encoder.fit_transform(df[neighbour_col])

df_dropped = df.drop(columns=neighbour_col)

df_encoded = pd.concat([df_dropped, transformed_columns], axis=1)
dump(df_encoded, "NN-encoder.joblib")
df_encoded

,agent_x,agent_y,target_x,target_y,hasItem,energy,charge,at_up_-1,at_up_0,at_up_1,...,at_left_3,at_left_4,at_left_5,at_right_-1,at_right_0,at_right_1,at_right_2,at_right_3,at_right_4,at_right_5
0,4,4,11,2,0,119,0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,1,6,5,0,119,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,9,7,3,2,0,119,0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,5,4,11,2,0,118,0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,2,1,6,5,0,118,0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103617,12,7,13,12,1,106,0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
103618,10,10,13,11,1,64,0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
103619,5,10,13,13,1,85,0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
103620,12,8,13,12,1,105,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# feature scaling

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_df = scaler.fit_transform(df_encoded)
dump(scaled_df, "NN-scaler.joblib")
scaled_df

array([[-0.36636164, -0.56075368,  0.90559216, ..., -0.14652636,
         1.53995641, -0.12008377],
       [-1.07223648, -1.31919109, -0.02553034, ..., -0.14652636,
        -0.64936903, -0.12008377],
       [ 0.81009644,  0.19768373, -0.58420385, ..., -0.14652636,
         1.53995641, -0.12008377],
       ...,
       [-0.13107002,  0.95612113,  1.27804116, ..., -0.14652636,
         1.53995641, -0.12008377],
       [ 1.51597128,  0.45049619,  1.27804116, ..., -0.14652636,
        -0.64936903, -0.12008377],
       [ 1.28067966,  0.95612113,  1.27804116, ..., -0.14652636,
         1.53995641, -0.12008377]], shape=(103622, 35))

In [5]:
# get train and test data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(scaled_df, all_labels, test_size=0.2, random_state=1)

# Train Model

In [6]:
from sklearn.neural_network import MLPClassifier


clf1 = MLPClassifier(hidden_layer_sizes=(20,20,20,20), max_iter=1000, random_state=1)
clf1.fit(X_train, y_train)
print("trained 1 complete")
clf2 = MLPClassifier(hidden_layer_sizes=(20,20,20,20,10), max_iter=1000, random_state=1)
clf2.fit(X_train, y_train)
print("trained 2 complete")
clf3 = MLPClassifier(hidden_layer_sizes=(20,20,20,20,10,10), max_iter=1000, random_state=1)
clf3.fit(X_train, y_train)
print("trained 3 complete")
clf4 = MLPClassifier(hidden_layer_sizes=(20,20,20,20,5), max_iter=1000, random_state=1)
clf4.fit(X_train, y_train)
print("trained 4 complete")

trained 1 complete
trained 2 complete
trained 3 complete
trained 4 complete


In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=1)
tree.fit(X_train, y_train)

In [ ]:
from sklearn.svm import SVC

svc = SVC(kernel="linear", C=0.025, random_state=42)
svc.fit(X_train, y_train)

In [ ]:
from sklearn.linear_model import Perceptron

ptr = Perceptron(tol=1e-3, random_state=0)
ptr.fit(X_train, y_train)

# Evaluation

In [7]:
from sklearn.metrics import accuracy_score

y_pred = clf1.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")

y_pred = clf2.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")

y_pred = clf3.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")

y_pred = clf4.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 95.37%
Accuracy: 95.29%
Accuracy: 95.40%
Accuracy: 95.38%


In [14]:
X_test

array([[ 1.04538805,  0.19768373,  0.16069416, ..., -0.14652636,
        -0.64936903, -0.12008377],
       [-0.36636164,  0.19768373,  0.90559216, ..., -0.14652636,
         1.53995641, -0.12008377],
       [ 0.33951321,  0.70330866, -0.77042835, ..., -0.14652636,
        -0.64936903, -0.12008377],
       ...,
       [ 1.04538805,  0.19768373, -0.21175484, ..., -0.14652636,
         1.53995641, -0.12008377],
       [ 0.57480482,  0.95612113, -1.14287735, ..., -0.14652636,
        -0.64936903, -0.12008377],
       [-1.30752809, -1.06637862, -1.14287735, ..., -0.14652636,
         1.53995641, -0.12008377]], shape=(20725, 35))

In [ ]:
y_pred = tree.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")

In [ ]:
y_pred = svc.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")

In [ ]:
y_pred = ptr.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")

# Save Model

In [10]:
# save simulated data!
from joblib import dump
dump(clf1, "NN-CLF-trainer.joblib")
# dump(tree, "NN-tree-trainer.joblib")
# dump(svc, "NN-svc-trainer.joblib")

['NN-CLF-trainer.joblib']

In [ ]:
from control.baseAgent import BaseAgent
import random

class NNAgent(BaseAgent):
    def __init__(self):
        super().__init__()

    def _choose_action(self, agent):
        encoded_move = clf1.predict([self._encode_state(agent)])
        return self.decode_move(agent, encoded_move)

    def _encode_state(self, agent):
        grid = self.model.gridSameShelf(self.model.gridWithAgent() , agent.orderItem)
        neighbours = self.model.get_neighbour_encoded(agent)
        return (
            agent.pos[0], agent.pos[1], agent.target_pos[0], agent.target_pos[1], # agent_x, agent_y, target_x, target_y
            neighbours[0], neighbours[1], neighbours[2], neighbours[3], agent.charge # at_up, at_down, at_left, at_right,agent
        )

    def decode_move(self, agent, encoded):
        movesID = self.model.get_neighbours(agent.pos)
        id = None
        match encoded[0]:
            case 0:
                id = 0
            case 1:
                id = 1
            case 2:
                id = 2
            case 3:
                id = 3
            case _:
                return agent.pos  # stay

        # validate before returning to ensure possible (if not, randomly find a valid move)
        movesID.append(agent.pos)
        while not self._validate_move(movesID[id], agent.orderItem):
            id = random.randint(0,4)
        return movesID[id]

    def _validate_move(self, pos_to, target_val):
        row = pos_to[0]
        col = pos_to[1]

        if row < 0 or row > self.model.grid_size[0]-1:
            return False
        if col < 0 or col > self.model.grid_size[1]-1:
            return False

        grid = self.model.gridSameShelf(self.model.gridWithAgent() , target_val)
        if grid[pos_to] == 1 or grid[pos_to] == 4:
            return False

        return True

In [ ]:
from model.model import Model
# from model.model import Model
from view.tkView import TkView
from timeit import default_timer as timer
from time import sleep
import random

ORDER_SIZE = 200
UNIT_TIME = 2400

# model & view
model = Model((15, 15), 5, ORDER_SIZE)
view = TkView(model)

# control
control = NNAgent()
control.initialise(model)

start = timer()
while model.timeStep < UNIT_TIME:
    control.moveAgents()
    view.render()
    print(f"Step {model.timeStep}: {model.printAgents()}")
    sleep(0.1)

    # break loop no agent working (all finished or all dead)
    orderDone = True
    for agent in model.agents:
        if agent.target_pos is not None:
            orderDone = False
    if orderDone:
        break
end = timer()


### Print out results ###

print(f"Total Real Time Taken: {end - start}")

print(f"Total Time Unit: {model.timeStep}")

deadAgentNum = 0
for agent in model.agents:
    if agent.energy == 0:
        deadAgentNum += 1

print(f"Total Dead agents: {deadAgentNum}")

orderDone = ORDER_SIZE-len(model.order_list)
orderPercent = (orderDone/ORDER_SIZE)*100
print(f"Total Orders dropped off: {orderDone}/{ORDER_SIZE}")
print(f"                           {orderPercent:.2f}%")

view.root.destroy()